In [48]:
# Preamble

# Hide simple warnings
import warnings; warnings.simplefilter('ignore')

# Check environment
import sys

_python = sys.executable
_pip = "%s -m pip -q" % _python

running_at_home = "anaconda3" in _python
running_in_env = "final" in _python

# Check executable
if running_at_home and not running_in_env:
  # I have a sweet custom config at home that I like.
  raise Exception(
      "No, stop. I want to run in my custom environment, please."
  )
else:
  # We're probably in a Google Colab.
  pass

# Show executable (for my own sanity)
_python

'c:\\Users\\buzzysin\\anaconda3\\envs\\final\\python.exe'

In [49]:
# Other setup
!{_pip} install -q --upgrade pip autopep8

# 1. Environment

In [50]:
## 1.1 Fetch YoloV8

# Install dependencies
!{_pip} install -r "https://raw.githubusercontent.com/ultralytics/ultralytics/main/requirements.txt"

# Install ultralytics (yolov8)
!{_pip} install ultralytics

In [51]:
# 1.2 Check CUDA Support

# Import ultralytics
import ultralytics

# Check to see if CUDA support is available
ultralytics.checks()


Ultralytics YOLOv8.0.51  Python-3.9.16 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
Setup complete  (16 CPUs, 13.9 GB RAM, 297.3/474.7 GB disk)


In [91]:
# 1.3 Setup and install loggers, etc.

from pathlib import Path

# Install comet_ml
!{_pip} install comet_ml

# 1.4 Download dataset

# Get the (not so) secret URL
with open("roboflow.txt", "r") as f:
  roboflow_url = f.readline().strip()

# Setup the directory structure
datasets_path = Path("../datasets")
data_path = Path("../data")

data_path.mkdir(parents=True, exist_ok=True)
datasets_path.mkdir(parents=True, exist_ok=True)

dataset_name = "circuits"

dataset_path = datasets_path / dataset_name
dataset_path.mkdir(parents=True, exist_ok=True)


In [92]:
# Images are stored in a zip file on Roboflow
!curl -SL {roboflow_url} > {dataset_name}.zip
!unzip -o {dataset_name}.zip -d ../datasets/{dataset_name}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   894  100   894    0     0   2290      0 --:--:-- --:--:-- --:--:--  2298

100 1647k  100 1647k    0     0  2297k      0 --:--:-- --:--:-- --:--:-- 2297k


Archive:  circuits.zip
 extracting: ../datasets/circuits/README.dataset.txt  
 extracting: ../datasets/circuits/README.roboflow.txt  
 extracting: ../datasets/circuits/data.yaml  
   creating: ../datasets/circuits/test/
   creating: ../datasets/circuits/test/images/
 extracting: ../datasets/circuits/test/images/image_019_png.rf.108272f397920947f23ac955b172a9ca.jpg  
 extracting: ../datasets/circuits/test/images/image_032_png.rf.16847e53afb48cf6f77df4d6c12f9b4d.jpg  
 extracting: ../datasets/circuits/test/images/image_033_png.rf.452983f51d62ffeeab4ffaea0872fee0.jpg  
 extracting: ../datasets/circuits/test/images/image_041_png.rf.117f8535c7c95f12c30bcf96dd289f47.jpg  
 extracting: ../datasets/circuits/test/images/image_044_png.rf.d336a198668a6435482e5cdd35c35354.jpg  
 extracting: ../datasets/circuits/test/images/image_045_png.rf.0966bdcdd12461f920049985c4df5a4c.jpg  
 extracting: ../datasets/circuits/test/images/image_049_png.rf.6a6287fc3afcbd9c0538da57c08bcba1.jpg  
 extracting: ../dat

In [94]:
def rmrf(path: "Path | str"):
  if isinstance(path, str):
    return rmrf(Path(path))

  if not path.exists():
    return

  if path.is_dir():
    for child in path.iterdir():
      rmrf(child)
    path.rmdir()
  else:
    path.unlink()

# Move the data to the correct location
data_path_bad = dataset_path / "data.yaml"

if data_path_bad.exists():
  data_path_bad.rename(data_path / "data.yaml")
  data_path_bad.unlink()

# Cleanup
try:
  rmrf(f"{dataset_name}.zip")
except:
  pass

# Remove dataset
# try:
#   rmrf(dataset_path)
# finally:
#   pass


# 2. YOLO (eugh...)

In [96]:
# 2.1 - Create a YOLOv8 model (from a pre-trained model)
from ultralytics import YOLO

# or yolov8s.pt, yolov8m.pt, yolov8l.pt, yolov8x.pt
model = YOLO("yolov8n.pt")


class my_config:
  def __init__(self, **kwargs):
    self.__dict__.update(kwargs)

  def __getitem__(self, key):
    return self.__dict__[key]

  def __setitem__(self, key, value):
    self.__dict__[key] = value

  def dict(self):
    return self.__dict__


config = my_config(
    data=f"{str(data_path / 'data.yaml')}",
    epochs=100,
    batch=16,
    imgsz=500,
    save=True,
    save_period=5,
    device="0",
)

# 2.2 - Train the model
results = model.train(**config.dict())
validated = model.val()

Ultralytics YOLOv8.0.51  Python-3.9.16 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=..\data\data.yaml, epochs=100, patience=50, batch=16, imgsz=500, save=True, save_period=5, cache=False, device=0, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, min_memory=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False